# Load and select data

In [2]:
import pandas as pd
import numpy as np


# --- Configuration ---
do_winsorize = False # Assuming this might be used later
lags = 1
reduce_banks = True
reduce_banks_to = 20

# --- Load Data ---
# Suppress warnings temporarily if needed, though fixing them is better
# warnings.filterwarnings('ignore', category=RuntimeWarning)
# warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

fred = pd.read_parquet('data/fred/macro_data_processed.parquet')
fdic = pd.read_parquet('data/fdic/fdic_data_processed.parquet')
yahoo = pd.read_parquet('data/yahoo/yahoo.parquet')

data_selected = fdic.merge(fred, on='date', how='left').merge(yahoo, on='date', how='left')
data_selected.set_index(['id', 'date'], inplace=True)

variables = [
    'gdp_qoq', 'cpi_qoq', 'sp500_qoq', 'corp_bond_spread',
    'cons_sentiment_qoq', 'unemployment', 'unemployment_diff', 'household_delinq', 'household_delinq_diff',
    'vix_qoq', 'spread_10y_3m',
    'log_total_assets', 'deposit_ratio', 'loan_to_asset_ratio',
    'equity_to_asset_ratio', 'trading_assets_ratio',
    'roa'
]

data_selected = data_selected[variables].copy()

## Test the selected data

In [3]:
print(data_selected.describe())

             gdp_qoq        cpi_qoq      sp500_qoq  corp_bond_spread  \
count  642383.000000  642383.000000  642383.000000     642383.000000   
mean        0.010699       0.006009       0.016421          0.025601   
std         0.013171       0.006112       0.083588          0.007509   
min        -0.082485      -0.022900      -0.225582          0.014600   
25%         0.007046       0.003699      -0.023018          0.020100   
50%         0.011541       0.006524       0.025686          0.025300   
75%         0.014300       0.009025       0.070272          0.029900   
max         0.087739       0.023894       0.199529          0.055800   

       cons_sentiment_qoq   unemployment  unemployment_diff  household_delinq  \
count       642383.000000  642383.000000      642383.000000     642383.000000   
mean            -0.001395       0.058664           0.000134          0.035795   
std              0.069664       0.018914           0.009757          0.012519   
min             -0.232919  

# Clean the data

In [4]:
data_cleaned = data_selected.copy()
initial_rows = len(data_cleaned)
cleaning_tracker = {}
print(f"Initial number of rows: {initial_rows}")

# Delete rows with zero total assets
rows_before = len(data_cleaned)
data_cleaned = data_cleaned[data_cleaned['log_total_assets'] > -np.inf]
rows_deleted = rows_before - len(data_cleaned)
cleaning_tracker['cleaning_ta_zero'] = rows_deleted
print(f"Deleted {rows_deleted} rows with zero or negative total assets.")

# Delete roa values smaller than -1 and greater than 1
rows_before = len(data_cleaned)
data_cleaned = data_cleaned[(data_cleaned['roa'] > -1) & (data_cleaned['roa'] < 1)]
rows_deleted = rows_before - len(data_cleaned)
cleaning_tracker['cleaning_roa_invalid'] = rows_deleted
print(f"Deleted {rows_deleted} rows with ROA < -1 or > 1.")

# Delete rows with loans to total assets ratio greater than or equal to 1
rows_before = len(data_cleaned)
data_cleaned = data_cleaned[data_cleaned['loan_to_asset_ratio'] < 1]
rows_deleted = rows_before - len(data_cleaned)
cleaning_tracker['cleaning_l2ta_ge_1'] = rows_deleted
print(f"Deleted {rows_deleted} rows with loans to total assets ratio >= 1.")

# Delete rows with equity to total assets ratio greater than or equal to 1
rows_before = len(data_cleaned)
data_cleaned = data_cleaned[data_cleaned['equity_to_asset_ratio'] < 1]
rows_deleted = rows_before - len(data_cleaned)
cleaning_tracker['cleaning_eq2ta_ge_1'] = rows_deleted
print(f"Deleted {rows_deleted} rows with equity to total assets ratio >= 1.")

# Delete rows with trading assets to total assets ratio < 0 or >= 1
rows_before = len(data_cleaned)
data_cleaned = data_cleaned[(data_cleaned['trading_assets_ratio'] < 1) & (data_cleaned['trading_assets_ratio'] >= 0)]
rows_deleted = rows_before - len(data_cleaned)
cleaning_tracker['cleaning_trada_invalid'] = rows_deleted
print(f"Deleted {rows_deleted} rows with trading assets to total assets ratio < 0 or >= 1.")

# Delete rows with equity to total assets ratio < 0 or >= 1
rows_before = len(data_cleaned)
data_cleaned = data_cleaned[(data_cleaned['equity_to_asset_ratio'] < 1) & (data_cleaned['equity_to_asset_ratio'] >= 0)]
rows_deleted = rows_before - len(data_cleaned)
cleaning_tracker['cleaning_eq2ta_invalid'] = rows_deleted
print(f"Deleted {rows_deleted} rows with equity to total assets ratio < 0 or >= 1.")



total_deleted = sum(cleaning_tracker.values())
print(f"\nTotal rows deleted: {total_deleted}")
print(f"Remaining rows: {len(data_cleaned)}")

# Optional: Display the tracker dictionary
# print(f"Cleaning tracker: {cleaning_tracker}")

# Assign the cleaned data back to data_processed for subsequent steps
# Note: The next cell (CELL INDEX 6) uses data_processed.
# If this cell is intended to be the primary cleaning step,
# rename data_cleaned to data_processed here.
print(f"\nShape after cleaning: {data_cleaned.shape}")

Initial number of rows: 644599
Deleted 0 rows with zero or negative total assets.
Deleted 96 rows with ROA < -1 or > 1.
Deleted 137 rows with loans to total assets ratio >= 1.
Deleted 276166 rows with equity to total assets ratio >= 1.
Deleted 0 rows with trading assets to total assets ratio < 0 or >= 1.
Deleted 300 rows with equity to total assets ratio < 0 or >= 1.

Total rows deleted: 276699
Remaining rows: 367900

Shape after cleaning: (367900, 17)


In [5]:
print(data_cleaned.describe())

             gdp_qoq        cpi_qoq      sp500_qoq  corp_bond_spread  \
count  367889.000000  367889.000000  367889.000000     367889.000000   
mean        0.010970       0.005876       0.033971          0.025775   
std         0.016178       0.005752       0.078573          0.007155   
min        -0.082485      -0.009760      -0.200011          0.014600   
25%         0.007737       0.003472       0.004366          0.021000   
50%         0.011773       0.005633       0.046941          0.025300   
75%         0.014546       0.007998       0.084737          0.029900   
max         0.087739       0.023894       0.199529          0.054900   

       cons_sentiment_qoq   unemployment  unemployment_diff  household_delinq  \
count       367889.000000  367889.000000      367889.000000     367889.000000   
mean             0.006087       0.062219          -0.000360          0.030213   
std              0.070819       0.023350           0.012645          0.013137   
min             -0.232919  

# Winsorize

In [6]:
def winsorize_dataframe(df, vars_to_winsorize, do_winsorize=False, lower_percentile=0.02, upper_percentile=0.98):
    """
    Winsorizes specified columns of a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.
        vars_to_winsorize (list): A list of column names to winsorize.
        do_winsorize (bool): If True, performs winsorization. Otherwise, returns a copy of the original DataFrame.
        lower_percentile (float): The lower percentile for clipping.
        upper_percentile (float): The upper percentile for clipping.

    Returns:
        pd.DataFrame: The winsorized (or original) DataFrame.
    """
    df_out = df.copy(deep=True)

    if not do_winsorize:
        return df_out

    def winsorize_series(series, lower_p, upper_p):
        lower_bound = series.quantile(lower_p)
        upper_bound = series.quantile(upper_p)
        return np.clip(series, lower_bound, upper_bound)

    print(f"Winsorizing columns at [{lower_percentile}, {upper_percentile}] percentiles...")
    for col in vars_to_winsorize:
        if col in df_out.columns:
            if pd.api.types.is_numeric_dtype(df_out[col]):
                print(f" - Winsorizing {col}")
                df_out[col] = winsorize_series(df_out[col], lower_percentile, upper_percentile)
            else:
                print(f" - Skipping non-numeric column {col}")
        else:
            print(f" - Skipping column {col} (not found in DataFrame)")
    print("Winsorization complete.")
    return df_out

# --- Apply Winsorization ---


# Define the columns to be winsorized in the features DataFrame.
# This is a simple heuristic based on column names. Adjust as needed.
accounting_columns = [col for col in data_cleaned.columns if 'ratio' in col or 'assets' in col or 'roa' in col]

# Winsorize features
data_winsorized = winsorize_dataframe(data_cleaned, accounting_columns, do_winsorize=do_winsorize, lower_percentile=0.02, upper_percentile=0.98)

print(f"Winsorized the following columns: {accounting_columns}")

Winsorized the following columns: ['log_total_assets', 'deposit_ratio', 'loan_to_asset_ratio', 'equity_to_asset_ratio', 'trading_assets_ratio', 'roa']


# Lag the data

In [7]:
data_processed = data_winsorized.copy()

# --- Feature Engineering ---


# Handle potential division by zero or NaNs if total_assets can be zero
data_processed['loan_to_asset_ratio'] = data_processed['loan_to_asset_ratio'].replace([np.inf, -np.inf], np.nan) # Or handle upstream
data_processed['term_spread_asset_ratio'] = data_processed['spread_10y_3m'] * data_processed['loan_to_asset_ratio']

# --- Create lagged variables efficiently ---

# Store original columns to iterate over (avoids lagging the lags)
original_cols = list(data_processed.columns)
lagged_features = [] # List to store the new lagged Series

print("Starting lag generation...")
# Loop through original columns
for col in original_cols:
     # Loop through desired lags
     for lag in range(1, lags + 1):
          # Calculate the lagged series
          # Use level='id' to group by the 'id' level of the MultiIndex
          lagged_series = data_processed.groupby(level='id')[col].shift(lag)
          # Rename the series uniquely
          lagged_series.name = f'{col}_lag{lag}'
          # Append the series to the list
          lagged_features.append(lagged_series)

print(f"Generated {len(lagged_features)} lagged features.")

# Concatenate all new lagged features at once
if lagged_features: # Check if the list is not empty
    print("Concatenating lagged features...")
    # This single concatenation is much more efficient
    data_processed = pd.concat([data_processed] + lagged_features, axis=1)
    print("Concatenation complete.")
else:
    print("No lagged features were generated.")

# --- Drop NaNs introduced by lagging ---
print(f"Shape before dropping NaNs: {data_processed.shape}")
print(f"Nr of banks before dropping NaNs: {data_processed.index.get_level_values('id').nunique()}")
data_processed.dropna(inplace=True) # Drop rows with NaNs
# Check the shape and number of banks after dropping NaNs
print(f"Shape after dropping NaNs: {data_processed.shape}")
print(f"Nr of banks after dropping NaNs: {data_processed.index.get_level_values('id').nunique()}")

# Min / max date
print(f"Min date: {data_processed.index.get_level_values('date').min()}")
print(f"Max date: {data_processed.index.get_level_values('date').max()}")

# Optional: Display info to verify
print("\nDataFrame Info after processing:")
data_processed.info()

print("\nSample of the processed DataFrame:")
print(data_processed.head())



Starting lag generation...
Generated 18 lagged features.
Concatenating lagged features...
Concatenation complete.
Shape before dropping NaNs: (367900, 36)
Nr of banks before dropping NaNs: 8342
Shape after dropping NaNs: (359547, 36)
Nr of banks after dropping NaNs: 8259
Min date: 157.0
Max date: 219.0

DataFrame Info after processing:
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 359547 entries, ('1000052', np.float64(157.0)) to ('999935', np.float64(219.0))
Data columns (total 36 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   gdp_qoq                       359547 non-null  float64
 1   cpi_qoq                       359547 non-null  float64
 2   sp500_qoq                     359547 non-null  float64
 3   corp_bond_spread              359547 non-null  float64
 4   cons_sentiment_qoq            359547 non-null  float64
 5   unemployment                  359547 non-null  float64
 6   unemployment_d

# Select features

In [8]:
X = data_processed.copy(deep=True)

# Define features: all variables except 'roa' with lags 1-4. Use variables list and add _lag1, _lag2, etc.
features = []  
for lag in range(1, lags + 1):
    features += [f"{var}_lag{lag}" for var in variables] 

# Select features
X = X[features].copy(deep=True)


# Restore default warning behavior if suppressed earlier
# warnings.filterwarnings('default')
# Select target
y = data_processed[['roa']].copy(deep=True) # Target is current ROA



In [9]:

# --- Check and Swap Index Levels ---
print("Checking and potentially swapping index levels to ('id', 'date')...")
def swap_index_levels(df):
    if isinstance(df.index, pd.MultiIndex) and len(df.index.levels) == 2:
        print(f"Current index names: {df.index.names}")
        if df.index.names == ['date', 'id']:
            print("Swapping levels from ('date', 'id') to ('id', 'date')...")
            df.index = df.index.swaplevel()
            print("Levels swapped. Sorting index...")
            df.sort_index(inplace=True)
            print("Index sorted.")
            print(f"New index names: {df.index.names}")
        elif df.index.names == ['id', 'date']:
            print("Index levels are already ('id', 'date'). Ensuring index is sorted...")
            df.sort_index(inplace=True)
            print("Index sorted.")
        else:
            print(f"Warning: Index names are {df.index.names}. Expected ['date', 'id'] or ['id', 'date']. Assuming ('id', 'date') is desired.")
            df.index.names = ['id', 'date'] # Assign names if None
            df.sort_index(inplace=True)
        print(f"Index dtypes after swap/sort: {[lvl.dtype for lvl in df.index.levels]}")
        print(f"First 5 index values:\n{df.index[:5]}")
    else:
        raise TypeError(f"DataFrame index must be a 2-level MultiIndex. Found index type: {type(df.index)}")
    return df

X = swap_index_levels(X)
y = swap_index_levels(y)

# Optional: Reduce number of banks to those with the highest total assets in 2010 and existing over the entire period
if reduce_banks:
    print("Reducing banks to those with the highest total assets in 2010 and existing over the entire period...")
    # Filter for banks existing in 2010
    banks_2010 = data_processed[data_processed.index.get_level_values('date') == '2010-01-01']
    # Get the top 10 banks by total assets
    top_banks = banks_2010.groupby(level='id')['log_total_assets'].max().nlargest(reduce_banks_to).index
    # Filter the original DataFrame for these banks
    X = X[X.index.get_level_values('id').isin(top_banks)]
    y = y[y.index.get_level_values('id').isin(top_banks)]
    print(f"Reduced to {len(top_banks)} banks.")



Checking and potentially swapping index levels to ('id', 'date')...
Current index names: ['id', 'date']
Index levels are already ('id', 'date'). Ensuring index is sorted...
Index sorted.
Index dtypes after swap/sort: [dtype('O'), dtype('float64')]
First 5 index values:
MultiIndex([('1000052', 157.0),
            ('1000052', 158.0),
            ('1000052', 159.0),
            ('1000052', 160.0),
            ('1000052', 161.0)],
           names=['id', 'date'])
Current index names: ['id', 'date']
Index levels are already ('id', 'date'). Ensuring index is sorted...
Index sorted.
Index dtypes after swap/sort: [dtype('O'), dtype('float64')]
First 5 index values:
MultiIndex([('1000052', 157.0),
            ('1000052', 158.0),
            ('1000052', 159.0),
            ('1000052', 160.0),
            ('1000052', 161.0)],
           names=['id', 'date'])
Reducing banks to those with the highest total assets in 2010 and existing over the entire period...
Reduced to 0 banks.


# Split the data

In [10]:


# --- Get dates AFTER cleaning NaNs and setting index ---
dates = X.index.get_level_values('date') # Ensure dates align with final X

# --- Time-based Split ---
train_mask = (dates.year <= 2020)
val_mask = (dates.year == 2022)
test_mask = (dates.year == 2024)

X_train = X[train_mask]
y_train = y[train_mask]
X_val = X[val_mask]
y_val = y[val_mask]
X_test = X[test_mask]
y_test = y[test_mask]

print(f"\nData Shapes:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_val:   {X_val.shape}, y_val:   {y_val.shape}")
print(f"X_test:  {X_test.shape}, y_test:  {y_test.shape}")


AttributeError: 'Index' object has no attribute 'year'

# Save the data

In [ ]:

# Save the final data
X_train.to_parquet("X_train.parquet")
y_train.to_parquet("y_train.parquet")
X_val.to_parquet("X_val.parquet")
y_val.to_parquet("y_val.parquet")
X_test.to_parquet("X_test.parquet")
y_test.to_parquet("y_test.parquet")

In [ ]:
# Run some sanity checks on splits
print(f"\nSanity checks on splits:")
print(f"X_train dates: {X_train.index.get_level_values('date').min()} to {X_train.index.get_level_values('date').max()}")
print(f"X_val dates:   {X_val.index.get_level_values('date').min()} to {X_val.index.get_level_values('date').max()}")
print(f"X_test dates:  {X_test.index.get_level_values('date').min()} to {X_test.index.get_level_values('date').max()}")
print(f"y_train dates: {y_train.index.get_level_values('date').min()} to {y_train.index.get_level_values('date').max()}")
print(f"y_val dates:   {y_val.index.get_level_values('date').min()} to {y_val.index.get_level_values('date').max()}")
print(f"y_test dates:  {y_test.index.get_level_values('date').min()} to {y_test.index.get_level_values('date').max()}")
# Check for any NaNs in the final datasets
print(f"\nChecking for NaNs in final datasets:")
print(f"X_train NaNs: {X_train.isna().sum().sum()}")
print(f"y_train NaNs: {y_train.isna().sum().sum()}")
print(f"X_val NaNs:   {X_val.isna().sum().sum()}")
print(f"y_val NaNs:   {y_val.isna().sum().sum()}")
print(f"X_test NaNs:  {X_test.isna().sum().sum()}")
print(f"y_test NaNs:  {y_test.isna().sum().sum()}")




Sanity checks on splits:
X_train dates: 2009Q2 to 2020Q4
X_val dates:   2022Q1 to 2022Q4
X_test dates:  2024Q1 to 2024Q4
y_train dates: 2009Q2 to 2020Q4
y_val dates:   2022Q1 to 2022Q4
y_test dates:  2024Q1 to 2024Q4

Checking for NaNs in final datasets:
X_train NaNs: 0
y_train NaNs: 0
X_val NaNs:   0
y_val NaNs:   0
X_test NaNs:  0
y_test NaNs:  0


In [ ]:
import pandas as pd

# --- Assuming X_train, X_val, X_test are already defined ---
# If not, uncomment and adapt the example creation/loading code:
# Example DataFrames (replace with your actual data)
# Make sure they have MultiIndex including 'date' or similar structure
# dates_train = pd.to_datetime(pd.date_range('2021-01-01', '2022-12-31', freq='QE'))
# dates_val = pd.to_datetime(pd.date_range('2023-01-01', '2023-12-31', freq='QE'))
# dates_test = pd.to_datetime(pd.date_range('2024-01-01', '2024-12-31', freq='QE'))
# ids = ['bank_a', 'bank_b']

# index_train = pd.MultiIndex.from_product([ids, dates_train], names=['id', 'date'])
# index_val = pd.MultiIndex.from_product([ids, dates_val], names=['id', 'date'])
# index_test = pd.MultiIndex.from_product([ids, dates_test], names=['id', 'date'])

# X_train = pd.DataFrame(np.random.rand(len(index_train), 3), index=index_train, columns=['feature_c', 'feature_a', 'feature_b'])
# X_val = pd.DataFrame(np.random.rand(len(index_val), 3), index=index_val, columns=['feature_c', 'feature_a', 'feature_b'])
# X_test = pd.DataFrame(np.random.rand(len(index_test), 3), index=index_test, columns=['feature_c', 'feature_a', 'feature_b'])
# import numpy as np # Don't forget this if using the example above

# --- Calculate Descriptive Statistics ---

datasets = {'Train': X_train, 'Validation': X_val, 'Test': X_test}
stats_to_compute = {
    'Mean': lambda df: df.mean(numeric_only=True),
    'Standard Deviation': lambda df: df.std(numeric_only=True),
    'Median': lambda df: df.median(numeric_only=True),
    'Minimum': lambda df: df.min(numeric_only=True),
    'Maximum': lambda df: df.max(numeric_only=True)
}

# Set display precision for floats
pd.set_option('display.float_format', lambda x: f'{x:.4f}')

# Get sorted feature names (assuming all datasets have the same features)
# Use X_train as reference, filter out non-numeric if necessary later
all_features = X_train.columns.tolist()
# Attempt to identify numeric features to avoid errors with .mean() etc.
# This is safer if non-numeric columns might exist
numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
sorted_features = sorted(numeric_features)


print("\n--- Descriptive Statistics ---")

for stat_name, stat_func in stats_to_compute.items():
    print(f"\n--- {stat_name} ---")
    summary_df = pd.DataFrame(index=sorted_features) # Ensure index is sorted features

    for ds_name, ds_df in datasets.items():
        if not ds_df.empty:
            # Calculate stat only for sorted numeric features
            # Use .reindex() to ensure consistent feature order and handle missing cols
            stats_series = stat_func(ds_df[sorted_features])
            summary_df[ds_name] = stats_series.reindex(sorted_features)
        else:
            # Add a column of NaNs if the dataset is empty
             summary_df[ds_name] = pd.NA

    # Display the table for the current statistic
    # Drop rows where all values are NaN (might happen if a feature was non-numeric)
    print(summary_df.dropna(how='all'))


# Reset display options if needed elsewhere
# pd.reset_option('display.float_format')


--- Descriptive Statistics ---

--- Mean ---
                             Train  Validation    Test
cons_sentiment_qoq_lag1     0.0072     -0.0692  0.0012
corp_bond_spread_lag1       0.0276      0.0203  0.0162
cpi_qoq_lag1                0.0042      0.0201  0.0066
deposit_ratio_lag1          0.6986      0.7785  0.7467
equity_to_asset_ratio_lag1  0.1334      0.1256  0.1322
gdp_qoq_lag1                0.0087      0.0238  0.0123
household_delinq_diff_lag1 -0.0008      0.0013  0.0007
household_delinq_lag1       0.0323      0.0178  0.0318
loan_to_asset_ratio_lag1    0.5946      0.5309  0.5614
log_total_assets_lag1      17.5515     18.1985 18.2341
roa_lag1                    0.0024      0.0035  0.0030
sp500_qoq_lag1              0.0333     -0.0395  0.0771
spread_10y_3m_lag1          0.0196      0.0136 -0.0094
trading_assets_ratio_lag1   0.0120      0.0104  0.0107
unemployment_diff_lag1      0.0004     -0.0040  0.0012
unemployment_lag1           0.0677      0.0378  0.0395
vix_qoq_lag1       